In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import kurtosis, skew
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scikit_posthocs as sp
from scipy.stats import wilcoxon
from matplotlib.patches import Patch
import pyarrow as pa
# import pyarrow.parquet as pq
import fastparquet
import random
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import statistics
from collections import Counter
from collections import OrderedDict
import statsmodels.stats.multicomp as mc
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
sns.set_style("whitegrid")

In [2]:
tempTopDir = '/Volumes/Britt_SSD/ReDoEvals3/UserStudy/'
filePath = os.path.join(tempTopDir, 'GP_Main.parquet')

filePath = os.path.join(tempTopDir, 'likertSlim_corrected.parquet')
likertSlim = pd.read_parquet(filePath)

# Path to save visuals
save_path = '/Users/byrdsmyth/iCloud/Documents/School/Thesis/Stimuli/'

likertSlim['visuals'] = np.where(likertSlim['visuals']=="Visuals","Charts",
                        np.where(likertSlim['visuals']=="Blur_Plus_Vis","Blur+Charts",likertSlim['visuals']))


In [3]:

likertSlim['visuals'] = np.where(likertSlim['visuals']=="Visuals","Charts",
                        np.where(likertSlim['visuals']=="Blur_Plus_Vis","Blur+Charts",likertSlim['visuals']))

df = likertSlim.copy()

fnt = 24
fnt2 = 19
fnt3 = 15

rSz = (12,7)
sqSz = (12,12)

AL = 0.26

saveFigs = False

boringBG = 'gold'

In [4]:
# drop particpants who have not been accepted (rejected or timed out)
pID_removed1 = likertSlim[(likertSlim['status']!='APPROVED')].ParticipantPublicID.unique()
print(len(likertSlim))
# drop these given row
# indexes from dataFrame
likertSlim = likertSlim[~likertSlim['ParticipantPublicID'].isin(pID_removed1)].copy()
print(len(likertSlim))
# tmp_slimDF = slimDF[~slimDF['ParticipantPublicID'].isin(pID_removed1)].copy()
# print(tmp_slimDF.status.unique())
# print(pID_removed1)
    
# tmp_demoDF_L = demoDF_L[~demoDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_demoDF_S = demoDF_S[~demoDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_consentDF_L = consentDF_L[~consentDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_consentDF_S = consentDF_S[~consentDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_aiDF_L = aiDF_L[~aiDF_L['ParticipantPublicID'].isin(pID_removed1)].copy()
# tmp_aiDF_S = aiDF_S[~aiDF_S['ParticipantPublicID'].isin(pID_removed1)].copy()

# tmp_taskDF = taskDF[~taskDF['ParticipantPublicID'].isin(pID_removed1)].copy()

# print(tmp_taskDF[tmp_taskDF['ParticipantPublicID']==pID_removed1[-3]])

219
219


In [5]:
df = likertSlim.copy()

for vis in df.visuals.unique():
    print(str(vis) + " " + str(len(df[df['visuals']==vis])))

Blur+Charts 59
Charts 51
Blurred 53
Video 56


In [6]:
vis = sorted(df['visuals'].unique())
print(vis)

['Blur+Charts', 'Blurred', 'Charts', 'Video']


In [7]:
# function to make statsmodel into dataframe
def results_summary_to_dataframe(results):
    '''take the result of an statsmodel results table and transforms it into a dataframe'''
    print(dir(results))
    pvals = results.pvalues
    coeff = results.params
    conf_lower = results.conf_int()[0]
    conf_higher = results.conf_int()[1]

    results_df = pd.DataFrame({"pvals":pvals,
                               "coeff":coeff,
                               "conf_lower":conf_lower,
                               "conf_higher":conf_higher
                                })

    #Reordering...
    results_df = results_df[["coeff","pvals","conf_lower","conf_higher"]]
    return results_df

In [8]:
def negative_bold(val):
    bold = 'bold' if val < 0 else ''
    return 'font-weight: %s' % bold

In [9]:


# temp = likertSlim.copy()

# temp['pred1_corrected'] = temp['predictions1Correct']-(temp['predictionsHalfTotal']-temp['predictions1Correct'])/(5-1)
# print('pred1_corrected')
# print(temp['pred1_corrected'].value_counts())
# print(temp['predictions1Correct'].value_counts())

# temp['pred2_corrected'] = temp['predictions2Correct']-(temp['predictionsHalfTotal']-temp['predictions2Correct'])/(5-1)
# print('pred2_corrected')
# print(temp['pred1_corrected'].value_counts())
# print(temp['predictions1Correct'].value_counts())

# temp['reg1_corrected'] = temp['regions1Correct']-(temp['regionsHalfTotal']-temp['regions1Correct'])/(8-1)
# print('reg1_corrected')
# print(temp['reg1_corrected'].value_counts())
# print(temp['regions1Correct'].value_counts())

# temp['reg2_corrected'] = temp['regions2Correct']-(temp['regionsHalfTotal']-temp['regions2Correct'])/(8-1)
# print('reg2_corrected')
# print(temp['reg2_corrected'].value_counts())
# print(temp['regions2Correct'].value_counts())

# temp['level1_corrected'] = temp['level1Correct']-(temp['levelHalfTotal']-temp['level1Correct'])/(4-1)
# print('level1_corrected')
# print(temp['level1_corrected'].value_counts())
# print(temp['level1Correct'].value_counts())

# temp['level2_corrected'] = temp['level2Correct']-(temp['levelHalfTotal']-temp['level2Correct'])/(4-1)
# print('level2_corrected')
# print(temp['level2_corrected'].value_counts())
# print(temp['level2Correct'].value_counts())

# temp['ghosts1_corrected'] = temp['ghosts1Correct']-(temp['ghostsHalfTotal']-temp['ghosts1Correct'])/(4-1)
# print("ghosts1")
# print(temp['ghosts1_corrected'].value_counts())
# print(temp['ghosts1Correct'].value_counts())

# temp['ghosts2_corrected'] = temp['ghosts2Correct']-(temp['ghostsHalfTotal']-temp['ghosts2Correct'])/(4-1)
# print("ghosts2")
# print(temp['ghosts2_corrected'].value_counts())
# print(temp['ghosts2Correct'].value_counts())


# temp['comp_corrected'] = temp['comparisonsCorrect']-(temp['comparisonsTotal']-temp['comparisonsCorrect'])/(2-1)
# print("Comp")
# print(temp['comp_corrected'].value_counts())
# print(temp['comparisonsCorrect'].value_counts())


# temp['Qs1_Corrected'] = temp['pred1_corrected'] + temp['reg1_corrected'] + temp['level1_corrected'] + \
#                         temp['ghosts1_corrected']

# temp['Qs2_Corrected'] = temp['pred2_corrected'] + temp['reg2_corrected'] + temp['level2_corrected'] + \
#                         temp['ghosts2_corrected']

# temp['1_2_correctedPerc'] = ((temp['Qs1_Corrected']+temp['Qs2_Corrected'])/(temp['QsPart_Total']*2))*100

# temp['Qs1_CorrectedPerc'] = (temp['Qs1_Corrected']/(temp['predictionsHalfTotal']+temp['regionsHalfTotal']+\
#                                                   temp['levelHalfTotal']+temp['ghostsHalfTotal']))*100

# temp['Qs2_CorrectedPerc'] = (temp['Qs2_Corrected']/(temp['predictionsHalfTotal']+temp['regionsHalfTotal']+\
#                                                   temp['levelHalfTotal']+temp['ghostsHalfTotal']))*100

# temp['comp_correctedPerc'] = (temp['comp_corrected']/temp['comparisonsTotal'])*100
# temp['tot_correctedPerc'] = ((temp['comp_corrected']+temp['Qs1_Corrected']+temp['Qs2_Corrected'])/20)*100

# print(temp['Qs1_CorrectedPerc'].value_counts())
# print(temp['Qs2_CorrectedPerc'].value_counts())
# print(temp['comp_correctedPerc'].value_counts())


In [10]:
print(temp['QsPart_Total'])

NameError: name 'temp' is not defined

In [ ]:
sns.set_style('whitegrid')
cp = sns.color_palette('tab10',4)
sns.set_palette(cp)

fig, axes = plt.subplots(nrows=2,ncols=1,figsize=rSz,sharex=True)

g = sns.pointplot('tot_correctedPerc','visuals', zorder=10, data=temp, order=["Charts", "Blur+Charts", "Video","Blurred"], \
                  alpha=1,join=False, ci=95, color='black',capsize=0.6, ax=axes[0])
g.set_title("Distribution of Performance by Explanation Condition",fontsize=fnt)

g = sns.stripplot('tot_correctedPerc','visuals',data=temp, zorder=0, jitter=0.37, \
                  order=["Charts", "Blur+Charts", "Video","Blurred"],alpha=AL, s=9, ax=axes[0])
# g.set_xlim((0,90))
g.set_xlabel("")
g.set_ylabel("")

for vis in ["Charts", "Blur+Charts", "Video","Blurred"]:
    tips_agg = temp[temp['visuals']==vis]
    i=sns.kdeplot(data=tips_agg,x='tot_correctedPerc',bw_adjust=0.95,ax=axes[1])
    i.set_xlabel("Average Percent Correct", fontsize=fnt2)
    
#     i.set_xlim((0,90))
axes[0].xaxis.grid(False) # Hide the horizontal gridlines
axes[1].xaxis.grid(False) # Show the vertical gridlines
axes[0].yaxis.grid(False) # Hide the horizontal gridlines
axes[1].yaxis.grid(False) # Show the vertical gridlines

legend_elements = [Patch(facecolor='dodgerblue', edgecolor='dodgerblue',
                         label='Charts'),
                   Patch(facecolor='orange', edgecolor='orange',
                         label='Blur+Charts'),
                   Patch(facecolor='green', edgecolor='g',
                         label='Video'),
                  Patch(facecolor='firebrick', edgecolor='firebrick',
                         label='Blurred')]

fig.legend(handles=legend_elements, bbox_to_anchor= (0.72, -0.001), ncol=4)

plt.tight_layout()
if saveFigs:
    filename = "Distribution_ofPerformance_byExplanationCondition_Normed.png"
    filepath = os.path.join(save_path, filename)
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()



In [ ]:
sns.set_style('whitegrid')
cp = sns.color_palette('tab10',4)
sns.set_palette(cp)

fig, axes = plt.subplots(nrows=1,ncols=1,figsize=rSz)

df['tot_correctedPerc'] = pd.to_numeric(df['tot_correctedPerc'])
# g = sns.pointplot('totPerc','visuals', zorder=10, data=df, order=["Charts", "Blur+Charts", "Video","Blurred"], \
#                   alpha=1,join=False, ci=95, color='black',capsize=0.6, ax=axes[0])
# g.set_title("Distribution of Performance by Explanation Condition",fontsize=fnt)

# g = sns.stripplot('totPerc','visuals',data=df, zorder=0, jitter=0.37, \
#                   order=["Charts", "Blur+Charts", "Video","Blurred"],alpha=AL, s=9, ax=axes[0])
# g.set_xlim((0,90))
# g.set_xlabel("")
# g.set_ylabel("")

for vis in ["Charts", "Blur+Charts", "Video","Blurred"]:
    tips_agg = df[df['visuals']==vis]
    i=sns.kdeplot(data=tips_agg,x='tot_correctedPerc',bw_adjust=0.95,ax=axes)
    i.set_xlabel("Average Percent Correct", fontsize=fnt2)
    
#     i.set_xlim((0,100))
    i.set_title("Distribution of Performance by Explanation Condition",fontsize=fnt)
# axes[0].xaxis.grid(False) # Hide the horizontal gridlines
axes.xaxis.grid(True) # Show the vertical gridlines
# axes[0].yaxis.grid(False) # Hide the horizontal gridlines
axes.yaxis.grid(True) # Show the vertical gridlines

legend_elements = [Patch(facecolor='dodgerblue', edgecolor='dodgerblue',
                         label='Charts'),
                   Patch(facecolor='orange', edgecolor='orange',
                         label='Blur+Charts'),
                   Patch(facecolor='green', edgecolor='g',
                         label='Video'),
                  Patch(facecolor='firebrick', edgecolor='firebrick',
                         label='Blurred')]

fig.legend(handles=legend_elements, bbox_to_anchor= (0.72, -0.001), ncol=4)

plt.tight_layout()
if saveFigs:
    filename = "Distribution_ofPerformance_byExplanationCondition_CurvesOnly.png"
    filepath = os.path.join(save_path, filename)
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sns.set_palette(cp)
df = temp.copy()
# histograms of comp versus ag
fig, axes = plt.subplots(nrows=3,ncols=1,figsize=rSz, sharex=True)

g = sns.pointplot(data=df, x="comp_correctedPerc",y='visuals', ax=axes[0],capsize=.7, order=["Charts", "Blur+Charts", "Video","Blurred"], palette=cp)
# g.set_xlim(40,85)
g.set_title('Part 3')
g.set_xlabel("")
g.set_ylabel("All",fontsize=fnt2)
g.set_title("")

g = sns.pointplot(data=df[df['difficulty']=="Easy"], x="comp_correctedPerc",y='visuals',ax=axes[1],capsize=.7, order=["Charts", "Blur+Charts", "Video","Blurred"], palette=cp)
# g.set_xlim(40,85)
g.set_title('Part 3')
g.set_xlabel("")
g.set_ylabel("Easy",fontsize=fnt2)
g.set_title("")

g = sns.pointplot(data=df[df['difficulty']=="Difficult"], x="comp_correctedPerc",y='visuals',ax=axes[2],capsize=.7, order=["Charts", "Blur+Charts", "Video","Blurred"], palette=cp)
# g.set_xlim(40,85)
g.set_title('Part 3')
g.set_xlabel("Performance (%)",fontsize=fnt2)
g.set_ylabel("Difficult",fontsize=fnt2)
g.set_title("")

fig.suptitle("Performance on Part 3: Comparisons",fontsize=fnt)
axes[0].xaxis.grid(True) # Hide the horizontal gridlines
axes[1].xaxis.grid(True) # Show the vertical gridlines
axes[2].xaxis.grid(True) # Hide the horizontal gridlines

legend_elements = [Patch(facecolor='dodgerblue', edgecolor='dodgerblue',
                         label='Charts'),
                   Patch(facecolor='orange', edgecolor='orange',
                         label='Blur+Charts'),
                   Patch(facecolor='green', edgecolor='g',
                         label='Video'),
                  Patch(facecolor='firebrick', edgecolor='firebrick',
                         label='Blurred')]

fig.legend(handles=legend_elements, bbox_to_anchor= (0.73, -0.001), ncol=4)

plt.tight_layout()
if saveFigs:
    filename = "Performance_onP3_Normed.png"
    filepath = os.path.join(save_path, filename)
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
sns.set_palette('tab10')
plt.figure(figsize=(8,8))
# tmp = likertSlim.reset_index()[['ParticipantPublicID','tot1Perc','tot2Perc','agTotPerc', \
#                                 'compPerc','totPerc']].melt('ParticipantPublicID', var_name='cols', \
#                                     value_name='vals')
# print(tmp)
# sns.pointplot(data=tmp, y='vals', x='cols', join=False, ci=100, orient='v')
# # plt.show()
# # print(tmp)
# # plt.figure(figsize=(15,4))
# sns.stripplot(data=tmp, y='vals', x='cols', orient='v',jitter=0.6, dodge=True)


j = 0.35
al = 0.4
sZ = 7
cS = 0.4

fig, axes = plt.subplots(nrows=4,ncols=1,figsize=rSz, sharex=True, sharey=False)

tmp = temp.reset_index()[['ParticipantPublicID','Qs1_CorrectedPerc']].melt('ParticipantPublicID', var_name='cols', \
                                    value_name='vals')
plt.figure(figsize=(10,4))
g1 = sns.stripplot(data=tmp, x='vals', y='cols', orient='h',jitter=j, alpha=al, s=sZ, ax=axes[0], dodge=True, color=boringBG, zorder=0)
g1 = sns.pointplot(data=tmp, x='vals', y='cols', orient='h',jitter=j,capsize=cS, alpha=al, s=sZ, ax=axes[0], dodge=True, color='black', zorder=10)
axes[0].set_xlabel("")
axes[0].set_ylabel("")
axes[0].set_yticklabels(["Part 1 %"], fontsize=fnt2)
# plt.show()

tmp = temp.reset_index()[['ParticipantPublicID','Qs2_CorrectedPerc']].melt('ParticipantPublicID', var_name='cols', \
                                    value_name='vals')
plt.figure(figsize=(15,4))
sns.stripplot(data=tmp, x='vals', y='cols', orient='h',jitter=j, alpha=al, s=sZ, ax=axes[1], dodge=True, color=boringBG, zorder=0)
sns.pointplot(data=tmp, x='vals', y='cols', orient='h',jitter=j, alpha=al,capsize=cS, s=sZ, ax=axes[1], dodge=True, color='black', zorder=10)
axes[1].set_xlabel("")
axes[1].set_ylabel("")
axes[1].set_yticklabels(["Part 2 %"], fontsize=fnt2)

tmp = temp.reset_index()[['ParticipantPublicID','1_2_correctedPerc']].melt('ParticipantPublicID', var_name='cols', \
                                    value_name='vals')
plt.figure(figsize=(15,4))
sns.stripplot(data=tmp, x='vals', y='cols', orient='h',jitter=j, alpha=al, s=sZ, ax=axes[2], dodge=True, color=boringBG, zorder=0)
sns.pointplot(data=tmp, x='vals', y='cols', orient='h',jitter=j, alpha=al,capsize=cS, s=sZ, ax=axes[2], dodge=True, color='black', zorder=10)
axes[2].set_xlabel("")
axes[2].set_ylabel("")
axes[2].set_yticklabels(["Parts 1&2 %"], fontsize=fnt2)

tmp = temp.reset_index()[['ParticipantPublicID','comp_correctedPerc']].melt('ParticipantPublicID', var_name='cols', \
                                    value_name='vals')
plt.figure(figsize=(15,4))
sns.stripplot(data=tmp, x='vals', y='cols', orient='h',jitter=j, alpha=al, s=sZ, ax=axes[3], dodge=True, color=boringBG, zorder=0)
sns.pointplot(data=tmp, x='vals', y='cols', orient='h',jitter=j, alpha=al,capsize=cS, s=sZ, ax=axes[3], dodge=True, color='black', zorder=10)
axes[3].set_ylabel("")
axes[3].set_xlabel("Performance (%)", fontsize=fnt2)
axes[3].set_yticklabels(["Part 3 %"], fontsize=fnt2)

# tmp = likertSlim.reset_index()[['ParticipantPublicID','totPerc']].melt('ParticipantPublicID', var_name='cols', \
#                                     value_name='vals')
# plt.figure(figsize=(15,4))
# sns.stripplot(data=tmp, x='vals', y='cols', orient='h',jitter=j, alpha=al, s=sZ, ax=axes[4], dodge=True, color='purple')
# sns.pointplot(data=tmp, x='vals', y='cols', orient='h',jitter=j, alpha=al, s=sZ, ax=axes[4], dodge=True, color='black')
# axes[4].set_xlabel("")
# axes[4].set_ylabel("")
# axes[4].set_yticklabels(["All %"])

axes[0].set_title("Participant Performance in Different Parts of the Survey", fontsize=fnt)

# plt.tight_layout()
if saveFigs:
    filename = "ParticipantPerformance_inDifferentParts_of_theSurvey_Normed.png"
    filepath = os.path.join(save_path, filename)
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:

cp = sns.color_palette('tab10',4)
sns.set_palette(cp)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=rSz)

df = temp.copy()
df['visuals'] = np.where(df['visuals']=="Visuals","Charts",
                        np.where(df['visuals']=="Blur_Plus_Vis","Blur+Charts",df['visuals']))
b_val = 0.98
# b_val = 1
for vis in ["Charts", "Blur+Charts", "Video","Blurred"]:
    tmp = df[df['visuals']==vis]
    tp = "step"
    tmp1 = tmp[['agTotPerc','visuals']]
    g = sns.kdeplot(data=tmp,x='1_2_correctedPerc',bw_adjust=b_val, ax=axes[0,0])
    axes[0,0].set_title("Parts 1 & 2",fontsize=fnt3)
    axes[0,0].set_ylabel("Performance",fontsize=fnt)
    axes[0,0].set_xlabel("Percent Correct")
    axes[0,0].set_xlim((-75,150))
    axes[0,0].set_ylim((0,0.035))
    
    g = sns.kdeplot(data=tmp,x='likertMode',bw_adjust=b_val, ax=axes[1,0])
    axes[1,0].set_ylabel("Perception",fontsize=fnt)
    axes[1,0].set_title("")
    axes[1,0].set_xlabel("Likert Response Mode")
    axes[1,0].set_ylim((0,0.6))
    
    g = sns.kdeplot(data=tmp,x='comp_correctedPerc',bw_adjust=b_val, ax=axes[0,1])
    axes[0,1].set_title("Part 3",fontsize=fnt3)
    axes[0,1].set_ylabel("")
    axes[0,1].set_xlabel("Percent Correct")
    axes[0,1].set_xlim((-75,150))
    axes[0,1].set_ylim((0,0.017))
    
    g = sns.kdeplot(data=tmp,x='CompResponseMode',bw_adjust=b_val, ax=axes[1,1])
    axes[1,1].set_title("")
    axes[1,1].set_xlabel("Likert Response Mode")
    axes[1,1].set_ylabel("")
    axes[1,1].set_ylim((0,0.6))

legend_elements = [Patch(facecolor='dodgerblue', edgecolor='dodgerblue',
                         label='Charts'),
                   Patch(facecolor='orange', edgecolor='orange',
                         label='Blur+Charts'),
                   Patch(facecolor='green', edgecolor='g',
                         label='Video'),
                  Patch(facecolor='firebrick', edgecolor='firebrick',
                         label='Blurred')]

fig.legend(handles=legend_elements, bbox_to_anchor= (0.69, -0.0008), ncol=4)
fig.suptitle("Participant Performance & Perception in Different Contexts",fontsize=fnt+3)

plt.tight_layout()
if saveFigs:
    filename = "ParticipantPerformancePerceptionCurvesDifferentContexts_Normed.png"
    filepath = os.path.join(save_path, filename)
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df = temp.copy()
cp = sns.color_palette('tab10',4)
sns.set_palette(cp)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=rSz)
j = 0.31
al = 0.56
sZ=9

g = sns.stripplot('likertMode_Z','visuals',data=df, order=["Charts", "Blur+Charts", "Video","Blurred"],zorder=0,ax=axes[1,0],jitter=j,alpha=AL,s=sZ)
# g.set_xlim((0.5,5.5))
g.set_xlabel("")

df['totPerc'] = pd.to_numeric(df['totPerc'])
g = sns.pointplot('likertMode_Z','visuals',data=df, order=["Charts", "Blur+Charts", "Video","Blurred"],zorder=10,ax=axes[1,0],join=False, ci=95, color='black',capsize=0.7)
g.set_xlabel("Normalized Likert Response Mode")
g.set_ylabel("Perception", fontsize=fnt2)
g.set_xlim((-2.8,2))





g = sns.stripplot('CompResponseMode_Z','visuals',data=df,zorder=0, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[1,1],jitter=j,alpha=AL,s=sZ)
# g.set_xlim((0.5,5.5))
g.set_xlabel("")

df['totPerc'] = pd.to_numeric(df['totPerc'])
g = sns.pointplot('CompResponseMode_Z','visuals',data=df,zorder=10, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[1,1],join=False, ci=95, color='black',capsize=0.7)
g.set_xlabel("Normalized Likert Response Mode")
g.set_xlim((-2.8,2))
g.set_ylabel("")




g = sns.stripplot('1_2_correctedPerc','visuals',data=df,zorder=0, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[0,0],jitter=j,alpha=AL,s=sZ)
g.set_xlim((-30,105.5))
g.set_xlabel("")

df['totPerc'] = pd.to_numeric(df['totPerc'])
g = sns.pointplot('1_2_correctedPerc','visuals',data=df,zorder=10, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[0,0],join=False, ci=95, color='black',capsize=0.7)
g.set_xlabel("Percent Correct")
# g.set_xlim((0,100))
g.set_ylabel("Performance", fontsize=fnt2)
g.set_title("Parts 1 & 2",fontsize=fnt3)


g = sns.stripplot('comp_correctedPerc','visuals',data=df,zorder=0, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[0,1],jitter=j,alpha=AL,s=sZ)
g.set_xlim((-30,105.5))
g.set_xlabel("")

df['totPerc'] = pd.to_numeric(df['totPerc'])
g = sns.pointplot('comp_correctedPerc','visuals',data=df,zorder=10, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[0,1],join=False, ci=95, color='black',capsize=0.7)
g.set_xlabel("Percent Correct")
g.set_ylabel("")
g.set_title("Part 3",fontsize=fnt3)

legend_elements = [Patch(facecolor='dodgerblue', edgecolor='dodgerblue',
                         label='Charts'),
                   Patch(facecolor='orange', edgecolor='orange',
                         label='Blur+Charts'),
                   Patch(facecolor='green', edgecolor='g',
                         label='Video'),
                  Patch(facecolor='firebrick', edgecolor='firebrick',
                         label='Blurred')]

fig.legend(handles=legend_elements, bbox_to_anchor= (0.73, -0.007), ncol=4)


fig.suptitle("Distribution of Performance and Perception by Explanation",fontsize=fnt)
plt.tight_layout()

if saveFigs:
    filename = "PerformancePerceptionJitterPlotsDifferentContexts_Normed.png"
    filepath = os.path.join(save_path, filename)
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df = temp.copy()
cp = sns.color_palette('tab10',4)
sns.set_palette(cp)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(13,6))
j = 0.31
al = 0.56
sZ=9

# g = sns.stripplot('likertMode_Z','visuals',data=df, order=["Charts", "Blur+Charts", "Video","Blurred"],zorder=0,ax=axes[1,0],jitter=j,alpha=AL,s=sZ)
# # g.set_xlim((0.5,5.5))
# g.set_xlabel("")

# df['totPerc'] = pd.to_numeric(df['totPerc'])
# g = sns.pointplot('likertMode_Z','visuals',data=df, order=["Charts", "Blur+Charts", "Video","Blurred"],zorder=10,ax=axes[1,0],join=False, ci=95, color='black',capsize=0.7)
# g.set_xlabel("Normalized Likert Response Mode")
# g.set_ylabel("Perception", fontsize=fnt2)
# g.set_xlim((-2.8,2))





# g = sns.stripplot('CompResponseMode_Z','visuals',data=df,zorder=0, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[1,1],jitter=j,alpha=AL,s=sZ)
# # g.set_xlim((0.5,5.5))
# g.set_xlabel("")

# df['totPerc'] = pd.to_numeric(df['totPerc'])
# g = sns.pointplot('CompResponseMode_Z','visuals',data=df,zorder=10, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[1,1],join=False, ci=95, color='black',capsize=0.7)
# g.set_xlabel("Normalized Likert Response Mode")
# g.set_xlim((-2.8,2))
# g.set_ylabel("")




g = sns.stripplot('1_2_correctedPerc','visuals',data=df,zorder=0, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[0],jitter=j,alpha=AL,s=sZ)
g.set_xlim((-30,105.5))
g.set_xlabel("")

df['totPerc'] = pd.to_numeric(df['totPerc'])
g = sns.pointplot('1_2_correctedPerc','visuals',data=df,zorder=10, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[0],join=False, ci=95, color='black',capsize=0.3)
g.set_xlabel("Percent Correct")
# g.set_xlim((0,100))
g.set_ylabel("Performance", fontsize=fnt2)
g.set_title("Parts 1 & 2",fontsize=fnt3)


g = sns.stripplot('comp_correctedPerc','visuals',data=df,zorder=0, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[1],jitter=j,alpha=AL,s=sZ)
g.set_xlim((-30,105.5))
g.set_xlabel("")

df['totPerc'] = pd.to_numeric(df['totPerc'])
g = sns.pointplot('comp_correctedPerc','visuals',data=df,zorder=10, order=["Charts", "Blur+Charts", "Video","Blurred"],ax=axes[1],join=False, ci=95, color='black',capsize=0.3)
g.set_xlabel("Percent Correct")
g.set_ylabel("")
g.set_title("Part 3",fontsize=fnt3)

legend_elements = [Patch(facecolor='dodgerblue', edgecolor='dodgerblue',
                         label='Charts'),
                   Patch(facecolor='orange', edgecolor='orange',
                         label='Blur+Charts'),
                   Patch(facecolor='green', edgecolor='g',
                         label='Video'),
                  Patch(facecolor='firebrick', edgecolor='firebrick',
                         label='Blurred')]

fig.legend(handles=legend_elements, bbox_to_anchor= (0.73, -0.007), ncol=4)


fig.suptitle("Distribution of Performance by Explanation",fontsize=fnt)
plt.tight_layout()

if saveFigs:
    filename = "PerformancePerceptionJitterPlotsDifferentContexts_Normed.png"
    filepath = os.path.join(save_path, filename)
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    fig.savefig(filepath, dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=rSz)
df['totPerc'] = pd.to_numeric(df['totPerc'])


g = sns.regplot('likertMode_All_Z', 'tot_correctedPerc', data=df, color="black", ax=axes)
g.set_title("Participant Perception and Performance on All Questions", fontsize=fnt)
g.set_xlabel("Perception", fontsize=fnt2)
g.set_ylabel("Performance", fontsize=fnt2)
plt.tight_layout()

if saveFigs:
    filename = "CorrelationPlots_Normed.png"
    filepath = os.path.join(save_path, filename)
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    fig.savefig(filepath, dpi=300,facecolor='white', bbox_inches='tight')
plt.show()

In [ ]:
# legend_elements = [Patch(facecolor='dodgerblue', edgecolor='dodgerblue',
#                          label='Charts'),
#                    Patch(facecolor='orange', edgecolor='orange',
#                          label='Blur+Charts'),
#                    Patch(facecolor='green', edgecolor='g',
#                          label='Video'),
#                   Patch(facecolor='firebrick', edgecolor='firebrick',
#                          label='Blurred')]

colors = ['dodgerblue','orange','green','firebrick']

fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(12,5), sharey=True, sharex=True)
axes = axes.ravel()
axes[0].set_xlim((df['likertMode_Z'].min(),df['likertMode_Z'].max()))

for v_index, vis in enumerate(["Charts", "Blur+Charts", "Video","Blurred"]):

    thisDF = df[df['visuals']==vis].copy()
    sns.regplot('likertMode_All_Z', 'tot_correctedPerc', data=thisDF, ax=axes[v_index], color=colors[v_index], truncate=False)
    axes[v_index].set_title(vis, fontsize=fnt3)
    axes[v_index].set_xlabel("")
    axes[v_index].set_ylabel("")
    
# axes[0].set_ylabel("Parts 1 & 2", fontsize=fnt3)
# axes[4].set_ylabel("Part 3", fontsize=fnt3)

fig.suptitle("Participant Perception and Performance by Explanation Condition", fontsize=fnt)
fig.supylabel("Performance", fontsize=fnt)
fig.supxlabel("Perception", fontsize=fnt2)

plt.tight_layout()

if saveFigs:
    filename = "CorrelationPlots_ByExplanation_AllTogether.png"
    filepath = os.path.join(save_path, filename)
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    fig.savefig(filepath, dpi=300,facecolor='white', bbox_inches='tight')
plt.show()

In [ ]:
likertSlim = temp.copy()
tempTopDir = '/Volumes/Britt_SSD/ReDoEvals3/UserStudy/'
filePath = os.path.join(tempTopDir, 'GP_Main.parquet')

In [ ]:
# Write Main Task DF to parquet:
filePath = os.path.join(tempTopDir, 'likertSlim.parquet')
# mainDF.to_parquet(path=filePath,compression='brotli')

table = pa.Table.from_pandas(likertSlim, safe=False)
# Parquet with Brotli compression
pq.write_table(table, filePath, compression='BROTLI')

# filePath = os.path.join(tempTopDir, 'GP_Main.csv')
# mainDF.to_csv(filePath)